In [ ]:
from os import makedirs
from os import listdir as lsdir
from os.path import join as pjoin
from os.path import exists as exist
from pathlib import Path
import glob
import sys
import pandas as pd
import shutil

import obspy
from obspy.clients.fdsn import Client
from obspy import read,read_inventory,read_events
from obspy.core import Stream,UTCDateTime
from obspy.geodetics.base import gps2dist_azimuth
from obspy.taup import TauPyModel
from obspy.signal.trigger import classic_sta_lta,plot_trigger,recursive_sta_lta,trigger_onset
from obspy.imaging.spectrogram import spectrogram
from obspy.signal.polarization import polarization_analysis
from obspy.signal.rotate import rotate2zne
from obspy.core import event

import instaseis

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import MultipleLocator as Mlc

In [ ]:
cat = read_events("quakes_xml/WafrQuakes90_21.xml")
Final_eq =  pd.read_csv('Output/Input_SSMTI/MTInputCatalog/event_summary.csv')     
Final_eq[['origin','S_corr','P_corr']]

In [ ]:
final_Pcorr = [0,0,0,0,0,0,0,0,0,-1,0,0,0,12.7,0]
final_Scorr = [-1,0,-20,0,0,0,0,0,21.8,-1,-6,0,0,-16,0]
P_corr_final_list = []
S_corr_final_list = []

#### Polarization Analyisis to confirm PZ & ST phases for selected Data

In [ ]:
t_pre_test = [30, 30]
t_post_test = [30, 30]
 
for eq,row in Final_eq.iloc[0:].iterrows():
    print(eq)

    ev_index = row['WAcat_index']
    event = cat[ev_index]
    print(event.origins[0].time)
    
    folder = row['folders']
    sta_selected = row['station']
    
    if sta_selected in ["IM.TOA0","IM.TOC2","IM.TOC4","IM.TOC6"]:
        day = row['origin'][:10].replace("-","_") # file format "2006_02_22"
        filename = "TORD_" + day + "_00:00:00.00000_"
        TORD_folder= "/scratch/tolugboj_lab/Prj13_WAfr_seismicity/2_Data/vDECWafr_read/%s*"%(filename)
        folderpath = glob.glob(TORD_folder)
        Tfile = folderpath[0][65:] +".wfdisc"
        TORD_file = pjoin(folderpath[0],Tfile) 
        print(TORD_file)
    else:
        datadir = "data/events/%s"%folder
        print(datadir)
        

    P_corr=row['P_corr']
    S_corr=row['S_corr']
    
    P_corr_final = P_corr + final_Pcorr[eq]
    S_corr_final = S_corr + final_Scorr[eq]
    
    P_corr_final_list.append(P_corr_final)
    S_corr_final_list.append(S_corr_final)
    
    P_S_arr=row['Arrival_PS'].strip("[]")
    P_S_arr=P_S_arr.split(",")
    
    freqmin= row['Freqmin']
    freqmax = row['Freqmax']
    freqmin_low = 0.1
    freqmax_low = 0.5
    freqmin_mid = 0.5
    freqmax_mid = 1.8
    
    corners = 3
   
    dist=row['epi_distance_deg']
    az=row['azimuths']
    baz=row['back_azimuths']

    #event info
    orig_time = event.origins[0].time
    location = row['country']
    mag = str(row['mag'])+" "+ row['magtype']


    if sta_selected in ["IM.TOA0","IM.TOC2","IM.TOC4","IM.TOC6"]:
            try:
                inv_tord = obspy.read_inventory("Inventories_xml/TORD.xml")
                inv=inv_tord
                beam = read(TORD_file)
                st = beam.select(station=sta_selected[3:])
                sta_meta = inv[0].select(station=sta_selected[3:],channel="BH*")
                azimuths=[]
                dips=[]
                
                
                #Remove Instrument Response
                for tr in st:                 
                    channelstr = tr.stats.channel
                    if channelstr == "BHZ":
                       
                        tr.stats.network = 'IM'
                        tr.stats.channel = 'BHZ'
                        tr.attach_response(inv)
                        
                        tr.detrend(type="linear")
                        tr.remove_response( 
                           pre_filt=(0.1,0.3,10,20), 
                           output="DISP", 
                           zero_mean=True) # correct to displacement
                        
                    elif channelstr == "BH1":
                        tr.stats.network = 'IM'
                        tr.stats.channel = 'BHZ'
                        tr.attach_response(inv)
                        
                        tr.detrend(type="linear")
                        tr.remove_response( 
                           pre_filt=(0.1,0.3,10,20), 
                           output="DISP", 
                           zero_mean=True) # correct to displacement
                        tr.stats.channel = 'BH1'
                        
                    else:
                        tr.stats.network = 'IM'
                        tr.stats.channel = 'BHZ'
                        tr.attach_response(inv)
                        
                        tr.detrend(type="linear")
                        tr.remove_response( 
                           pre_filt=(0.1,0.3,10,20), 
                           output="DISP", 
                           zero_mean=True) # correct to displacement
                        tr.stats.channel = 'BH2'
                    
                 
                #Rotation
                for ch in [0,1,2]:
                    chan=sta_meta.stations[0].channels[ch]
                    azimuths.append(chan.azimuth)
                    dips.append(chan.dip)
                strot = rotate2zne(st[0], azimuths[0], dips[0],st[1], azimuths[1], dips[1],st[2], azimuths[2], dips[2], inverse=False)
     
                for tr in st:
                    channelstr = tr.stats.channel
                    if channelstr == "BHZ":
                        tr.data = strot[0]
                    elif channelstr == "BH1":
                        tr.stats.channel = "BHN"
                        tr.data = strot[1]
                    else:
                        tr.stats.channel = "BHE"
                        tr.data = strot[2]
        
        
        
        
            except Exception as e:
                exception_type, exception_object, exception_traceback = sys.exc_info()
                line_number = exception_traceback.tb_lineno 
                print(line_number)
                print(exception_object)
    else: 
        
            path_to_inventory = pjoin(datadir, ("stations/*"))
            stat_list = lsdir(path_to_inventory[:-1])
            stat_list.sort()
         
            inv_iris = obspy.read_inventory(path_to_inventory)
            inv=inv_iris
            path_to_waveform = pjoin(datadir,("waveforms/%s*.mseed")%sta_selected)
            st = obspy.read(path_to_waveform)
            st.detrend(type="linear")
            st.remove_response(inventory=inv, 
                       pre_filt=(0.1,0.3,10,20), 
                       output="DISP", 
                       zero_mean=True) # correct to displacement

    #P and S time
    ptt = float(P_S_arr[0])
    stt = float(P_S_arr[1])
    

    # Detrend and remove instrument response
    try:
        
        st.detrend(type="linear")
        st.detrend(type="demean") 


        #save raw data for final Inversion
#         folder_wav = "Output/Input_SSMTI/InputWaveforms/%s"%folder
#         Path(folder_wav).mkdir(parents=True,exist_ok=True)     
#         for tr in st:
#             tr.write(pjoin(folder_wav,"%s.mseed"%tr.id),format="mseed")
#             print("file saved")

        
        
        for freq in zip([freqmin,freqmin_mid,freqmin_low],[freqmax,freqmax_mid,freqmax_low],[0,4,8]):

            # Data for Polarization Analysis
            st_filtPA = st.copy()
            st_filt_pol_Full =  st_filtPA.filter("bandpass",freqmin=freq[0],freqmax=freq[1],corners=corners,zerophase=True)

            # Data Rotated 
            st_ = st.copy()
            
            for tr in st_:
                tr.stats.distance = dist
                tr.stats.back_azimuth = baz
            st_.rotate(method="NE->RT")
            
            st_filtP = st_.copy()
            st_filtS = st_.copy()
            st_filtP = st_filtP.filter("bandpass",freqmin=freq[0],freqmax=freq[1],corners=corners,zerophase=True)
            st_filtS = st_filtS.filter("bandpass",freqmin=freq[0],freqmax=freq[1],corners=corners,zerophase=True)


            #P
            BHZwavP = st_filtP.select(channel="BHZ")
            BHRwavP = st_filtP.select(channel="BHR")
            BHTwavP = st_filtP.select(channel="BHT")

            BHZwavP = BHZwavP.slice(starttime=orig_time + ptt - t_pre_test[0],
                                   endtime= orig_time + ptt + t_post_test[0],
                                   )
            BHRwavP = BHRwavP.slice(starttime=orig_time + ptt - t_pre_test[0],
                                   endtime= orig_time + ptt + t_post_test[0],
                                   )

            BHTwavP = BHTwavP.slice(starttime=orig_time + ptt - t_pre_test[0],
                                   endtime= orig_time + ptt + t_post_test[0],
                                   )


            #S
            BHZwavS = st_filtS.select(channel="BHZ")
            BHRwavS = st_filtS.select(channel="BHR")
            BHTwavS = st_filtS.select(channel="BHT")

            BHZwavS = BHZwavS.slice(starttime=orig_time + stt - t_pre_test[1],
                                   endtime= orig_time + stt + t_post_test[1],
                                   )
            BHRwavS = BHRwavS.slice(starttime=orig_time + stt - t_pre_test[1],
                                   endtime= orig_time + stt + t_post_test[1],
                                   )

            BHTwavS = BHTwavS.slice(starttime=orig_time + stt - t_pre_test[1],
                                   endtime= orig_time + stt + t_post_test[1],
                                   )                        


            # Polarization Input
            Stream_Full = st_filt_pol_Full.slice(starttime=orig_time + ptt - 130,
                                                 endtime= orig_time + stt + 330)

            ymin = -1
            ymax = 1

            trZP = BHZwavP.traces[0].data
            trZP /= max(abs(trZP))
            trRP = BHRwavP.traces[0].data
            trRP /= max(abs(trRP))
            trTP = BHTwavP.traces[0].data
            trTP /= max(abs(trTP))

            trZS = BHZwavS.traces[0].data
            trZS /= max(abs(trZS))
            trRS = BHRwavS.traces[0].data
            trRS /= max(abs(trRS))
            trTS = BHTwavS.traces[0].data
            trTS /= max(abs(trTS))



            plt.style.use('default')
            plt.rcParams["figure.figsize"] = (28,40)

            #1
            ax1 = plt.subplot2grid((14, 2), (2+freq[2], 0), colspan=1,rowspan=2)

            timeZP = BHZwavP.traces[0].times()
            timeZP = timeZP - t_pre_test[0]
            ax1.plot(timeZP,trZP,color="blue",linewidth=4)
            ax1.plot(timeZP,trRP-2,color="dimgrey",linewidth=3)
            ax1.plot(timeZP,trTP-4,color="dimgrey",linewidth=3)
            ax1.text(-30,0.25,"P",fontsize = 50)
            ax1.text(-30,2,"[%s-%s] Hz"%(freq[0],freq[1]),fontsize=40)
            ax1.axvline(P_corr_final,color="g",linewidth=5)
            ax1.axvline(P_corr,color="k",ls="dashed",linewidth=5)
            #annotation
            xtc = P_corr + ((P_corr_final - P_corr)/2)
            if (P_corr_final - P_corr) >= 5 or (P_corr_final - P_corr) <=-5 :
                ax1.hlines(-3,P_corr,P_corr_final,lw=5,colors='black')
                ax1.vlines(P_corr,-3.29,-2.69,lw=10,colors='black')
                ax1.vlines(P_corr_final,-3.29,-2.69,lw=10,colors='black')
                ax1.text(xtc,-3.7,"$T_{C}$",fontsize=40,fontweight = 'bold',ha="center")
                        
            ax1.set_xlim(-t_pre_test[0],t_post_test[0])
            ax1.set_yticks([-4,-2,0])
            ax1.set_yticklabels(['T','R','Z'])
            ax1.set_xticks(range(-t_pre_test[0],t_post_test[0]+10,10))
            ax1.tick_params(axis="both", labelsize=35,direction="out",length=12)
            ax1.set_ylim(-5.0,1.0)
            for i in [0,2,4]:
                ax1.scatter(0.0, -0.5-i, marker="^",color="black",s=500)

            for axis in ['top','bottom','left','right']:
                ax1.spines[axis].set_linewidth(2)
            
            
            #2 
            ax2 = plt.subplot2grid((14, 2), (2+freq[2], 1), colspan=1,rowspan=2)

            timeZS = BHZwavS.traces[0].times()
            timeZS = timeZS - t_pre_test[1]
            ax2.plot(timeZS,trZS,color="dimgrey",linewidth=3)
            ax2.plot(timeZS,trRS-2,color="dimgrey",linewidth=3)
            ax2.plot(timeZS,trTS-4,color="maroon",linewidth=4)
            ax2.text(-29,0.25,"S",fontsize = 50)
            ax2.axvline(S_corr_final,color="g",linewidth=5)
            ax2.axvline(S_corr,color="k",ls="dashed",linewidth=5)
            #annotation
            xtc = S_corr + ((S_corr_final - S_corr)/2)
            if (S_corr_final - S_corr) >= 5  or (S_corr_final - S_corr) <= -5:
                ax2.hlines(-3,S_corr,S_corr_final,lw=5,colors='black',label="Arrival Time Correction (Tc)")
                ax2.vlines(S_corr,-3.29,-2.69,lw=10,colors='black')
                ax2.vlines(S_corr_final,-3.29,-2.69,lw=10,colors='black')
                ax2.text(xtc,-3.7,"$T_{C}$",fontsize=40,fontweight = 'bold',ha="center")
            
            ax2.set_xlim(-t_pre_test[1],t_post_test[1])
            ax2.set_yticks([-4,-2,0])
            ax2.set_yticklabels(['T','R','Z'])
            ax2.set_xticks(range(-t_pre_test[1],t_post_test[1]+10,10))
            ax2.tick_params(axis="both", labelsize=35,direction="out",length=12)
            ax2.set_ylim(-5.0,1.0)
            for i in [0,2,4]:
                ax2.scatter(0.0, -0.5-i, marker="^",color="black",s=500)
            
            for axis in ['top','bottom','left','right']:
                ax2.spines[axis].set_linewidth(2)

            # Pol. Analysis Full waveform 
            win_len= 10
            win_frac= 0.2

            #3
            #Parameters
            stream = Stream_Full
            frqlow= freq[0]
            frqhigh = freq[1]


            stime =orig_time + ptt - 130 + 0.05            
            etime =orig_time + stt + 330 - 0.05

            p_pa = polarization_analysis(stream , win_len, win_frac, 
                                  frqlow, frqhigh, stime, etime, 
                                  verbose=False, method='flinn', 
                                  var_noise=0.0, adaptive=True)


            timepol = []
            ReferenceTimeFP = orig_time + ptt
            timestamps = p_pa['timestamp']
            for i in timestamps:
                time_ = UTCDateTime(i) 
                time_ = time_ - ReferenceTimeFP  
                timepol.append(time_)
            timepol = np.array(timepol)


            for i, parameters in enumerate(['rectilinearity']): 
                ax3= plt.subplot2grid((14, 2), (4+freq[2], 0), colspan=2,rowspan=1)
                values = np.round(p_pa[parameters],2)
                ax3.plot(timepol,values,color='black',marker='o',markersize=5,linewidth=3)
                ax3.axvspan(xmin=-30,xmax=30,ymin=0,ymax=1,facecolor= "none",lw=2,edgecolor="black")
                ax3.text(-29,0.7,"P",fontsize = 35)
                ax3.scatter(stt-ptt,0.15,marker="^",color="black",label="Predicted (IASP91)",s=500)
                ax3.axvline(P_corr_final,color="g",linewidth=5)  
                ax3.axvline(P_corr,color="k",ls="dashed",label="Preliminary (STA/LTA)",linewidth=5)
          
                ax3.axvspan(xmin=stt-ptt-30,xmax=stt-ptt+30,ymin=0,ymax=1,facecolor= "none",lw=2,edgecolor="black")
                ax3.text(stt-ptt-29,0.7,"S",fontsize = 35)
                ax3.axvline(stt-ptt+S_corr_final,color="g",label="Estimated",linewidth=5)
                ax3.axvline(stt-ptt+S_corr,color="k",ls="dashed",linewidth=5)
                ax3.scatter(0.0,0.15,marker="^",color="black",s=500)
                
                if (S_corr_final - S_corr) >= 5  or (S_corr_final - S_corr) <= -5 or (P_corr_final - P_corr) >= 5  or (P_corr_final - P_corr) <= -5:
                    ax3.hlines(-3,1,2,lw=5,colors='black',label="Arrival Time Correction (Tc)")
                

                ax3.set_xlim(-30,stt-ptt+40)
                ax3.xaxis.set_minor_locator(Mlc(10))
                ax3.xaxis.set_major_locator(Mlc(30))
                ax3.set_yticks(np.arange(0.1,1.0,0.4))
                ax3.set_ylim(0.0,1.0)
                ax3.tick_params(axis="both", labelsize=35,direction="out",length=12)

                for pos in ['top','right']:
                    ax3.spines[pos].set_visible(False)
                for axis in ['bottom','left']:
                    ax3.spines[axis].set_linewidth(2)
                ax3.set_ylabel("%s"%parameters,fontsize=35)

            ax3.set_xlabel("Time(s)",fontsize=35)

        plt.tight_layout()
        plt.suptitle(" %s\n%s  Mag: %s\nStation: %s\n Distance: %s$^\circ$ Az: %s$^\circ$ Baz: %s$^\circ$"%(location,
                                                                        str(orig_time)[0:19],mag,sta_selected,
                                                                        round(dist,1),round(az,1),
                                                                        round(baz,1)),horizontalalignment ="center",fontsize=40)
        plt.legend(bbox_to_anchor=(0.87,21.8),ncol=1, prop={"size": 35},loc='center',)

        output ="Output/Figures/Phase_Corrections"
        plt.savefig(pjoin(
                     output,f"{folder}_Phase_Corrections.pdf"
                         ),

                     dpi=600,
                    )
        plt.show()
        #plt.close(fig) 
        

    except Exception as e:
            exception_type, exception_object, exception_traceback = sys.exc_info()
            line_number = exception_traceback.tb_lineno 
            print(line_number)
            print(exception_object)
            
            
            

## SNR  for corrected picks

In [ ]:
final_Pcorr2 = [0,0,0,0,0,0,0,0,0,-1,0,0,0,13.7,0]
final_Scorr2 = [-1,0,-20.5,0,0,0,0,0,21.8,-1,-4.5,0,0,-14.5,0]

t_pre_test = [20, 30]
t_post_test = [20, 20]

Psnr_list_corr=[]
Ssnr_list_corr=[]

plt.style.use('default')
plt.rcParams["figure.figsize"] = (20,10)
 
for eq,row in Final_eq.iloc[0:].iterrows():
    print(eq)
    ev_index = row["WAcat_index"]
    event = cat[ev_index]
    print(event.origins[0].time)
    
    folder = row['folders']
    sta_selected = row['station']


    
    baz = row['back_azimuths']
    dist = row['epi_distance_deg']

    
    P_corr=row['P_corr']
    S_corr=row['S_corr']
    
    P_corr_final = P_corr + final_Pcorr2[eq]
    S_corr_final = S_corr + final_Scorr2[eq]
    
    
    P_S_arr=row['Arrival_PS'].strip("[]")
    P_S_arr=P_S_arr.split(",")
    
    freqmin= row['Freqmin']
    freqmax = row['Freqmax']

    corners = 3
   
    epicentral_distance_degree=row['epi_distance_deg']
    az=row['azimuths']
    baz=row['back_azimuths']

    #event info
    orig_time = event.origins[0].time
    location = row['country']
    mag = str(row['mag'])+" "+ row['magtype']

            
    path_to_waveform = "Output/Input_SSMTI/InputWaveforms/%s/%s*.mseed"%(folder,sta_selected)
    st= obspy.read(path_to_waveform)

    #P and S time
    ptt = float(P_S_arr[0])
    stt = float(P_S_arr[1])
    

    for tr in st:
        tr.stats.distance = dist
        tr.stats.back_azimuth = baz
    st.rotate(method="NE->RT")


    # Apply filter 
    st_filt = st.copy()
    st_filt =st_filt.filter("bandpass",freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)

    #
    st_filtS = st_filt.copy()


    #P
    BHZwavP = st_filt.select(channel="BHZ")
    BHZwavP= BHZwavP.slice(starttime=orig_time + ptt - t_pre_test[0],
                           endtime= orig_time + ptt  + t_post_test[0],
                           )

    #S
    BHTwavS = st_filtS.select(channel="BHT")
    BHTwavS = BHTwavS.slice(starttime=orig_time + stt - t_pre_test[1],
                           endtime= orig_time + stt + t_post_test[1],
                           )

    trPZ = BHZwavP.traces[0].data
    trPZ /= max(abs(trPZ))


    trST = BHTwavS.traces[0].data
    trST /= max(abs(trST))

        
    # PZ & ST Phase Detection and Quality Analysis
    #trg = [4.5,3.5]
    STL= 10
    LTL =100

    cftP = recursive_sta_lta(trPZ,STL,LTL)



    cftS= recursive_sta_lta(trST,STL,LTL)


    deltaP = BHZwavP.traces[0].stats.delta
    nptsP = BHZwavP.traces[0].stats.npts
    tmaxP = int(nptsP * deltaP)
    xanP= t_pre_test[0]/deltaP
    xanP_corr = (t_pre_test[0] + P_corr)/deltaP
    xanP_corr_f = (t_pre_test[0] + P_corr_final)/deltaP
        

    deltaS =  BHTwavS.traces[0].stats.delta
    nptsS =  BHTwavS.traces[0].stats.npts
    tmaxS = int(nptsS * deltaS)
    xanS  = t_pre_test[1]/deltaS
    xanS_corr = (t_pre_test[1] + S_corr)/deltaS
    xanS_corr_f = (t_pre_test[1] + S_corr_final)/deltaS
    
    mininP = int(xanP_corr_f-50)
    maxinP = int(xanP_corr_f+60)
    select_cftP = max(cftP[mininP:maxinP])

    
    mininS = int(xanS_corr_f-50)
    maxinS = int(xanS_corr_f+60)
    select_cftS = max(cftS[mininS:maxinS])
    
    print(f"P_SNR: {select_cftP}")
    print(f"S_SNR: {select_cftS}")
    
    Psnr_list_corr.append(select_cftP)
    Ssnr_list_corr.append(select_cftS)
  
        
    #P
    ax1 = plt.subplot2grid((3,2), (1, 0), colspan=1,rowspan=1)
    
    ax1.plot(trPZ,'blue',lw=2)
    ax1.set_xlim(0,nptsP)
    ax1.scatter(xanP, -0.8, marker="^",color="black",s=500)    
    ax1.axvline(xanP_corr_f,color="g",label="Estimated",linewidth=2)
    ax1.axvline(xanP_corr,color="k",ls="dashed",linewidth=2)
    ax1.axvspan(xanP_corr_f-20,xanP_corr_f+60,color="grey")
    ax1.set_xticks(np.arange(0,nptsP,10/deltaP))
    ax1.set_xticklabels([])
    ax1.set_yticks(np.arange(-1,2,1))
    ax1.set_ylim(-1,1)
    ax1.tick_params(axis="both", labelsize=20,direction="out",length=12)
    for axis in ['top','bottom','left','right']:
            ax1.spines[axis].set_linewidth(2)
    
    ax3 = plt.subplot2grid((3,2), (2, 0), colspan=1,rowspan=1)
    ax3.plot(cftP,'blue',lw=2)
    ax3.scatter(xanP, 0.5, marker="^",color="black",s=500)
    ax3.axvline(xanP_corr_f,color="g",label="Estimated",linewidth=2)
    ax3.axvline(xanP_corr,color="k",ls="dashed",linewidth=2)
    ax3.axvspan(xanP_corr_f-20,xanP_corr_f+60,color="grey")
    ax3.set_yticks(np.arange(0,6,0.5))
    ax3.set_xlim(0,nptsP)
    ax3.set_xticks(np.arange(0,nptsP,10/deltaP))
    ax3.set_xticklabels(np.arange(-t_pre_test[0],(tmaxP+10)-t_post_test[0],10))
    ax3.tick_params(axis="both", labelsize=20,direction="out",length=12)
    ax3.set_ylabel("STA/LTA",fontsize=35)
    ax3.set_xlabel("Time(s)",fontsize=35)
    for axis in ['top','bottom','left','right']:
            ax3.spines[axis].set_linewidth(2)
    
    
    #S
    ax2 = plt.subplot2grid((3,2), (1, 1), colspan=1,rowspan=1)
    ax2.plot(trST,'maroon',lw=2)
    ax2.set_xlim(0,nptsS)
    ax2.scatter(xanS, -0.8, marker="^",color="black",s=500)
    ax2.axvline(xanS_corr_f,color="g",label="Estimated",linewidth=2)
    ax2.axvline(xanS_corr,color="k",ls="dashed",linewidth=2)
    ax2.axvspan(xanS_corr_f-20,xanS_corr_f+60,color="grey")
    ax2.set_xticks(np.arange(0,nptsS,10/deltaS)) 
    ax2.set_xticklabels([])
    ax2.set_yticks(np.arange(-1,2,1))
    ax2.set_ylim(-1,1)
    ax2.tick_params(axis="both", labelsize=20,direction="out",length=12)
    for axis in ['top','bottom','left','right']:
            ax2.spines[axis].set_linewidth(2)
    
    ax4 = plt.subplot2grid((3,2), (2, 1), colspan=1,rowspan=1)
    ax4.plot(cftS,'maroon',lw=2)
    ax4.scatter(xanS, 0.5, marker="^",color="black",s=500)
    ax4.axvline(xanS_corr_f,color="g",label="Estimated",linewidth=2)
    ax4.axvline(xanS_corr,color="k",ls="dashed",linewidth=2)
    ax4.axvspan(xanS_corr_f-20,xanS_corr_f+60,color="grey")
    ax4.set_yticks(np.arange(0,6,0.5))
    ax4.set_xlim(0,nptsS)
    ax4.set_xticks(np.arange(0,nptsS,10/deltaS))
    ax4.set_xticklabels(np.arange(-t_pre_test[1],(tmaxS+10)-t_post_test[1],10))
    ax4.tick_params(axis="both", labelsize=20,direction="out",length=12)
    ax4.set_ylabel("STA/LTA",fontsize=35)
    ax4.set_xlabel("Time(s)",fontsize=35)
    for axis in ['top','bottom','left','right']:
            ax4.spines[axis].set_linewidth(2)
            
            
    plt.tight_layout()
    plt.suptitle(" %s\n%s"%(location,
                            str(orig_time)[0:19]),
                            horizontalalignment="center",fontsize=40)
            
    plt.show()

In [ ]:
#SNR of Corrected Picks
indices = [2,10,13] # data where arrivals were corrected


for i in indices:
    if i in [2,13]:
        print(Final_eq["P_SNR"][i])
        Final_eq["P_SNR"][i] = Psnr_list_corr[i]
    print(Final_eq["S_SNR"][i])
    Final_eq["S_SNR"][i] = Ssnr_list_corr[i]
print("------")
for i in indices:
    print(Final_eq["P_SNR"][i])
    print(Final_eq["S_SNR"][i])

#### Final P and  S time correction

In [ ]:
Reviewed_eq = Final_eq
Reviewed_eq["P_corr"] = P_corr_final_list
Reviewed_eq["S_corr"] = S_corr_final_list

In [ ]:
Reviewed_eq=Reviewed_eq.drop([4,5,12])
Reviewed_eq.reset_index(drop=True, inplace=True)
Reviewed_eq

#### Final Eq Catalog


In [ ]:
#create new catalog
final_eq_catalog = obspy.core.event.Catalog()
WAMCA_input = Reviewed_eq

for eq,row in Reviewed_eq.iterrows():
    ev_index = row['WAcat_index']
    event = cat[ev_index]
    final_eq_catalog.append(event)
print(final_eq_catalog)
#Save Catalog
final_eq_catalog.write("./Output/Input_SSMTI/MTInputCatalog/WAMCA_Quakes.xml",format="QUAKEML")

#Update Summary table
#rows_to_modify =
#WAMCA_input


#WAMCA_input.drop(columns='WAcat_index',inplace=True)
WAMCA_input.to_csv('Output/Input_SSMTI/MTInputCatalog/WAMCA_input.csv',index=False)